In [63]:
import pandas as pd

In [64]:
import os
import pandas as pd

# folder = "/Users/luisfaria/Desktop/sEngineer/masters_SWEAI/2025-T2/T2-REM/assignments/Assessment2/sales"

# If in codespaces
folder = "/workspaces/masters-swe-ai/2025-T2/T2-REM/assignments/Assessment2/sales"

# List to hold dataframes
dataframes = []

# Read all Excel files and append to list
for file in os.listdir(folder):
    if file.endswith(".xlsx"):
        filepath = os.path.join(folder, file)
        df = pd.read_excel(filepath)
        dataframes.append(df)

# Concatenate all dataframes into one
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print("Data combined successfully.")
else:
    print("No Excel files found.")

# Optional: print the first few rows
combined_df.sample(5)

Data combined successfully.


,ID orçamento,Data orçamento,Status,Revenda,Data venda,Mês venda,Hora venda,Unidade,Consultor,Avaliador,...,Nome cliente,CPF cliente,ID cliente,Email do cliente,Telefone(s) do cliente,Data nascimento cliente,Profissão cliente,Cortesia?,Qtd itens orçamento,Unnamed: 27
23598,420688,2024-12-10 17:51:22,Finalizado,NÃO,2024-12-10 17:53:18,Dezembro,17:53:18,PRAIA GRANDE,Ranna de Jesus Santana,AVALIAÇÃO PRAIA GRANDE,...,Larissa de Oliveira Nascimento Melo,074.059.214-93,1008315.0,larissastefan@hotmail.com,13981882224,1988-11-22 00:00:00,Professor,Não,NaN,NaN
6308,430406,2025-02-06 19:15:03,Cancelado,NÃO,2025-02-06 19:16:34,Fevereiro,19:16:34,LAPA,Esther Geovanna Alves Santana Carvalho Barbosa,Julia Macena Dantas,...,Silmara Silva,080.028.408-95,1020452.0,ss.silmara@alsanner.com,11964550770,1966-07-13 00:00:00,Vendedor,Não,NaN,NaN
6307,430406,2025-02-06 19:15:03,Cancelado,NÃO,2025-02-06 19:16:34,Fevereiro,19:16:34,LAPA,Esther Geovanna Alves Santana Carvalho Barbosa,Julia Macena Dantas,...,Silmara Silva,080.028.408-95,1020452.0,ss.silmara@alsanner.com,11964550770,1966-07-13 00:00:00,Vendedor,Não,NaN,NaN
24462,421538,2024-12-16 12:05:56,Finalizado,NÃO,2024-12-16 13:38:50,Dezembro,13:38:50,ITAIM,Thais Silva Lima,NaN,...,Evellyn carolayne Rodrigues dos Santos,138.794.736-25,1009257.0,Evellyncarolayne@gmail.com,55489666140 / 11977109024,2001-07-10 00:00:00,Administrador (a),Não,NaN,NaN
12252,436470,2025-03-11 18:33:20,Finalizado,SIM,2025-03-11 18:34:48,Março,18:34:48,JARDINS,Lara Goncalo Aparicio,Simone Rocha Meireles,...,Felipe de Barros Souza,360.259.148-42,1022616.0,fefobarross@gmail.com,11982835803,1988-08-26 00:00:00,Ator,Não,NaN,NaN


In [65]:
df.columns

Index(['ID orçamento', 'Data orçamento', 'Status', 'Revenda', 'Data venda',
       'Mês venda', 'Hora venda', 'Unidade', 'Consultor', 'Avaliador',
       'Valor tabela', 'Valor desconto', 'Valor líquido', 'Grupo procedimento',
       'Procedimento', 'Quantidade', 'Valor tabela item',
       'Valor % desconto item', 'valor desconto item', 'Valor liquido item',
       'Fonte do cadastro do cliente', 'Nome cliente', 'CPF cliente',
       'ID cliente', 'Email do cliente', 'Telefone(s) do cliente',
       'Data nascimento cliente', 'Profissão cliente', 'Cortesia?'],
      dtype='object')

In [66]:
desired_columns = [
        'ID orçamento', 'Status','Data venda', 'Mês venda',
        'Unidade', 'Valor líquido', 'ID cliente']

df_filtered = combined_df[desired_columns]

# treating
df_filtered = df_filtered.loc[df_filtered['Status'] == 'Finalizado']
df_filtered['Valor líquido'] = df_filtered['Valor líquido'].fillna(0)
df_filtered.sort_values(by='Valor líquido', ascending=False)

# total sum of 'Valor líquido'
df_filtered['Valor líquido'].sum()



np.float64(17719676.38)

In [67]:
df_filtered['Data venda'] = pd.to_datetime(df_filtered['Data venda'])
df_filtered['Year'] = df_filtered['Data venda'].dt.year
df_filtered['Month'] = df_filtered['Data venda'].dt.month

In [76]:
# convert 'Valor líquido' to float
df_filtered['Valor líquido'] = df_filtered['Valor líquido'].astype(float)


df_filtered.groupby(['Unidade', 'Month', 'Year'])['Valor líquido'].sum().unstack(fill_value=0)

Year                       2024      2025
Unidade        Month                     
ALPHAVILLE     1           0.00  76386.00
               2           0.00  82787.04
               3           0.00  60769.91
               12      92290.08      0.00
BELO HORIZONTE 1      111643.77      0.00
...                         ...       ...
TUCURUVI       12     152590.93      0.00
VILA MASCOTE   1           0.00  94511.00
               2           0.00  60550.99
               3           0.00  50164.01
               12      84607.97      0.00

[88 rows x 2 columns]

In [77]:
df_tucuruvi = df_filtered.loc[df_filtered['Unidade'] == 'TUCURUVI']
df_tucuruvi.groupby(['Unidade', 'Month', 'Year'])['Valor líquido'].sum().unstack(fill_value=0)

Year                 2024       2025
Unidade  Month                      
TUCURUVI 1      187160.08  164754.90
         2           0.00  155763.51
         3           0.00  160833.49
         12     152590.93       0.00